Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from wordcloud import WordCloud
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

Data Collection

In [ ]:
train_data = pd.read_csv("train_data.txt", delimiter=':::', header = None, engine='python')
test_data  = pd.read_csv("test_data.txt", delimiter=':::', header = None, engine='python')
test_data_solution  = pd.read_csv("test_data_solution.txt", delimiter=':::' ,header = None ,engine='python')

df = pd.concat((train_data ,test_data_solution))
df.columns = ["id", "Title", "Genre", "Description"]

Data Preprocessing

In [ ]:
#drop duplicates
df.drop_duplicates(inplace = True)

#drop nan values
df.dropna(inplace = True)

In [ ]:
stopword = set(stopwords.words('english'))

def preprocessing(text):
    text = text.lower()

    text = re.sub(r'[^\w\s]', '', text)

    text = re.sub(r'[#@\$]', '', text)

    tokens = word_tokenize(text)

    lemmatizer  = WordNetLemmatizer()

    text = text.split()

    text = [word for word in text if word not in stopword]

    return " ".join(text)

In [ ]:
df["Despcription_clean"] =  df["Description"].apply(preprocessing)

Feature Engineering

In [ ]:
#dropping unnecessary columns
data = df.drop(["Title", "id"], axis = 1)

Model

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split

from sklearn.metrics import classification_report,confusion_matrix,ConfusionMatrixDisplay

In [ ]:
#Convert genre labels to numerical values for modeling
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
data['Genre_encoded'] = label_encoder.fit_transform(data['Genre'])
data['Genre_encoded']

class_names= list(label_encoder.classes_)

Train Test Split

In [ ]:
x = data["Despcription_clean"]
y = data["Genre"]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.5)

Vectorizing

In [ ]:
vectorize = CountVectorizer()
x_train1 = vectorize.fit_transform(x_train)
x_test1 =  vectorize.transform(x_test)

In [ ]:
mnb = MultinomialNB()
mnb.fit(x_train1 ,y_train)
print("Model Score on Training data",mnb.score(x_train1 ,y_train))
print("Model Score on Training data",mnb.score(x_test1 ,y_test))
y_pred = mnb.predict(x_test1)

print(classification_report(y_pred ,y_test))

Model Score on Training data 0.6389580681461804
Model Score on Training data 0.5252827125647979


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


               precision    recall  f1-score   support

      action        0.03      0.69      0.06        58
       adult        0.03      0.51      0.06        35
   adventure        0.03      0.52      0.06        52
   animation        0.00      0.00      0.00         0
   biography        0.00      0.00      0.00         0
      comedy        0.44      0.53      0.48      6217
       crime        0.00      0.00      0.00         0
 documentary        0.90      0.57      0.70     20874
       drama        0.83      0.46      0.59     24428
      family        0.00      1.00      0.01         2
     fantasy        0.00      0.00      0.00        13
   game-show        0.13      1.00      0.24        25
     history        0.00      0.00      0.00         0
      horror        0.30      0.74      0.42       854
       music        0.04      0.75      0.08        40
     musical        0.00      1.00      0.01         1
     mystery        0.00      0.00      0.00         0
        n

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
model = LogisticRegression()
model.fit(x_train1 ,y_train)
print("Model Score on Training data",model.score(x_train1 ,y_train))
print("Model Score on Training data",model.score(x_test1 ,y_test))
y_pred = model.predict(x_test1)
print(classification_report(y_pred ,y_test))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Model Score on Training data 0.9969745604811187
Model Score on Training data 0.5719187558802369
               precision    recall  f1-score   support

      action        0.32      0.42      0.37      1044
       adult        0.40      0.59      0.47       376
   adventure        0.24      0.39      0.30       474
   animation        0.16      0.38      0.22       203
   biography        0.01      0.05      0.01        39
      comedy        0.58      0.54      0.56      7966
       crime        0.09      0.20      0.12       231
 documentary        0.79      0.71      0.75     14590
       drama        0.69      0.56      0.62     16592
      family        0.17      0.35      0.23       391
     fantasy        0.10      0.30      0.15       108
   game-show        0.58      0.80      0.67       136
     history        0.02      0.14      0.04        42
      horror        0.59      0.61      0.60      2081
       music        0.52      0.64      0.57       599
     musical        0.0

In [ ]:
svm = LinearSVC()
svm.fit(x_train1 ,y_train)
print("Model Score on Training data",svm.score(x_train1 ,y_train))
print("Model Score on Training data",svm.score(x_test1 ,y_test))
y_pred = svm.predict(x_test1)
print(classification_report(y_pred ,y_test))

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Model Score on Training data 0.9994834615455569
Model Score on Training data 0.520652314276754
               precision    recall  f1-score   support

      action        0.28      0.32      0.30      1174
       adult        0.40      0.49      0.44       459
   adventure        0.23      0.31      0.26       580
   animation        0.16      0.26      0.20       296
   biography        0.01      0.02      0.02       135
      comedy        0.53      0.50      0.51      7870
       crime        0.09      0.13      0.10       350
 documentary        0.72      0.70      0.71     13611
       drama        0.61      0.54      0.57     15267
      family        0.17      0.25      0.21       548
     fantasy        0.11      0.16      0.13       215
   game-show        0.58      0.68      0.63       158
     history        0.03      0.06      0.04       108
      horror        0.56      0.52      0.54      2272
       music        0.49      0.57      0.53       633
     musical        0.08

In [ ]:
random_forest = RandomForestClassifier()

random_forest.fit(x_train1, y_train)
print("Random Forest - Train Score:",random_forest.score(x_train1, y_train))
print("Random Forest - Test Score:", random_forest.score(x_test1, y_test))

y_pred = random_forest.predict(x_test1)
print(classification_report(y_pred ,y_test))

Random Forest - Train Score: 0.9995203571494456
Random Forest - Test Score: 0.48949397679266515


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


               precision    recall  f1-score   support

      action        0.01      0.80      0.02        15
       adult        0.07      0.63      0.13        67
   adventure        0.07      0.78      0.13        74
   animation        0.00      1.00      0.00         1
   biography        0.00      0.00      0.00         1
      comedy        0.26      0.55      0.35      3492
       crime        0.00      1.00      0.01         2
 documentary        0.89      0.56      0.69     20709
       drama        0.85      0.41      0.55     28351
      family        0.02      0.86      0.03        14
     fantasy        0.00      0.00      0.00         0
   game-show        0.47      0.87      0.61       100
     history        0.00      1.00      0.01         1
      horror        0.12      0.78      0.21       323
       music        0.09      0.81      0.17        84
     musical        0.01      1.00      0.03         4
     mystery        0.00      0.00      0.00         0
        n

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Using TF-IDF Vectorizer

In [ ]:
vectorizer = TfidfVectorizer()
x_train2 = vectorize.fit_transform(x_train)
x_test2 =  vectorize.transform(x_test)

In [ ]:
mnb = MultinomialNB()
mnb.fit(x_train2 ,y_train)
print("Model Score on Training data",mnb.score(x_train2 ,y_train))
print("Model Score on Training data",mnb.score(x_test2 ,y_test))
y_pred = mnb.predict(x_test2)
print(classification_report(y_pred ,y_test))

Model Score on Training data 0.6389580681461804
Model Score on Training data 0.5252827125647979


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


               precision    recall  f1-score   support

      action        0.03      0.69      0.06        58
       adult        0.03      0.51      0.06        35
   adventure        0.03      0.52      0.06        52
   animation        0.00      0.00      0.00         0
   biography        0.00      0.00      0.00         0
      comedy        0.44      0.53      0.48      6217
       crime        0.00      0.00      0.00         0
 documentary        0.90      0.57      0.70     20874
       drama        0.83      0.46      0.59     24428
      family        0.00      1.00      0.01         2
     fantasy        0.00      0.00      0.00        13
   game-show        0.13      1.00      0.24        25
     history        0.00      0.00      0.00         0
      horror        0.30      0.74      0.42       854
       music        0.04      0.75      0.08        40
     musical        0.00      1.00      0.01         1
     mystery        0.00      0.00      0.00         0
        n

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
model = LogisticRegression()
model.fit(x_train2 ,y_train)
print("Model Score on Training data",model.score(x_train2 ,y_train))
print("Model Score on Training data",model.score(x_test2 ,y_test))
y_pred = model.predict(x_test2)
print(classification_report(y_pred ,y_test))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Model Score on Training data 0.9969745604811187
Model Score on Training data 0.5719187558802369
               precision    recall  f1-score   support

      action        0.32      0.42      0.37      1044
       adult        0.40      0.59      0.47       376
   adventure        0.24      0.39      0.30       474
   animation        0.16      0.38      0.22       203
   biography        0.01      0.05      0.01        39
      comedy        0.58      0.54      0.56      7966
       crime        0.09      0.20      0.12       231
 documentary        0.79      0.71      0.75     14590
       drama        0.69      0.56      0.62     16592
      family        0.17      0.35      0.23       391
     fantasy        0.10      0.30      0.15       108
   game-show        0.58      0.80      0.67       136
     history        0.02      0.14      0.04        42
      horror        0.59      0.61      0.60      2081
       music        0.52      0.64      0.57       599
     musical        0.0

In [ ]:
svm = LinearSVC()
svm.fit(x_train2 ,y_train)
print("Model Score on Training data",svm.score(x_train2 ,y_train))
print("Model Score on Training data",svm.score(x_test2 ,y_test))
y_pred = svm.predict(x_test2)
print(classification_report(y_pred ,y_test))

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Model Score on Training data 0.9994834615455569
Model Score on Training data 0.520560075267032
               precision    recall  f1-score   support

      action        0.28      0.32      0.30      1174
       adult        0.40      0.49      0.44       459
   adventure        0.23      0.30      0.26       581
   animation        0.16      0.26      0.19       296
   biography        0.01      0.02      0.02       135
      comedy        0.53      0.50      0.51      7865
       crime        0.09      0.13      0.10       350
 documentary        0.72      0.70      0.71     13612
       drama        0.61      0.54      0.57     15267
      family        0.17      0.25      0.21       548
     fantasy        0.11      0.16      0.13       215
   game-show        0.58      0.68      0.63       158
     history        0.03      0.06      0.04       108
      horror        0.56      0.52      0.54      2274
       music        0.49      0.57      0.53       632
     musical        0.08

In [ ]:
random_forest = RandomForestClassifier()
random_forest.fit(x_train2, y_train)

print("Random Forest - Train Score:",random_forest.score(x_train2, y_train))
print("Random Forest - Test Score:", random_forest.score(x_test2, y_test))

y_pred = random_forest.predict(x_test2)
print(classification_report(y_pred ,y_test))

Random Forest - Train Score: 0.9995203571494456
Random Forest - Test Score: 0.4907484273248842


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


               precision    recall  f1-score   support

      action        0.01      0.73      0.02        15
       adult        0.05      0.75      0.09        36
   adventure        0.09      0.71      0.15        95
   animation        0.00      0.00      0.00         0
   biography        0.00      0.00      0.00         1
      comedy        0.26      0.56      0.36      3492
       crime        0.00      1.00      0.01         2
 documentary        0.89      0.56      0.69     20754
       drama        0.86      0.41      0.55     28315
      family        0.02      0.88      0.03        16
     fantasy        0.00      0.00      0.00         0
   game-show        0.44      0.87      0.59        95
     history        0.00      0.00      0.00         0
      horror        0.11      0.75      0.19       313
       music        0.10      0.76      0.17        92
     musical        0.01      1.00      0.02         3
     mystery        0.00      0.00      0.00         0
        n

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
